In [206]:
using JuMP
using GLPK

In [213]:
model = Model(GLPK.Optimizer)

# decision variable
@variable(model, crude1 >= 0)
@variable(model, crude2 >= 0)

# output variable
@variable(model, butane >= 0)
@variable(model, petrol >= 0)
@variable(model, diesel >= 0)
@variable(model, heating >= 0)

# distillation variables
@variable(model, disbutane >= 0)
@variable(model, naphtha >= 0)
@variable(model, residue >= 0)
@variable(model, gasoil >= 0)

# reforming variable
@variable(model, refbutane >= 0)
@variable(model, reformate >= 0)

# cracking
@variable(model, crknaphtha >= 0)
@variable(model, crkgasoil >= 0)

# input petrol
@variable(model, petbutane >= 0)
@variable(model, petcrknaphtha >= 0)

# input heating oil
@variable(model, hocrknaphtha >= 0)
@variable(model, hocrkgasoil >= 0)
@variable(model, hogasoil >= 0)

# input diesel oil
@variable(model, dslcrknaphtha >= 0)
@variable(model, dslcrkgasoil >= 0)
@variable(model, dslgasoil >= 0)

# distillation constraint
@constraint(model, disbutane <= 0.03 * crude1 + 0.05 * crude2)
@constraint(model, naphtha <= 0.15 * crude1 + 0.2 * crude2)
@constraint(model, residue <= 0.15 * crude1 + 0.1 * crude2)
@constraint(model, gasoil <= 0.4 * crude1 + 0.35 * crude2)

# reforming constraint
@constraint(model, refbutane <= 0.15 * naphtha)
@constraint(model, reformate <= 0.85 * naphtha)

# cracking constraint
@constraint(model, crknaphtha <= 0.4 * residue)
@constraint(model, crkgasoil <= 0.35 * residue)

# balance
@constraint(model, crknaphtha == petcrknaphtha + hocrknaphtha + dslcrknaphtha)
@constraint(model, crkgasoil == hocrkgasoil + dslcrkgasoil)
@constraint(model, gasoil == hogasoil + dslgasoil)

# demand constraint
@constraint(model, butane >= 20000)
@constraint(model, petrol >= 40000)
@constraint(model, diesel >= 30000)
@constraint(model, heating >= 42000)

# raw material constraint
@constraint(model, crude1 <= 250000)
@constraint(model, crude2 <= 500000)

# capacities
@constraint(model, naphtha <= 30000) # reformer
@constraint(model, residue <= 40000) # cracking
@constraint(model, gasoil <= 50000) # desulfurization

# characteristics
@constraint(model, octane_value, 120 * petbutane + 100 * reformate + 74 * petcrknaphtha >= 94 * petrol)
@constraint(model, vapor_pressure, 60 * petbutane + 2.6 * reformate + 4.1 * petcrknaphtha <= 12.7 * petrol)
@constraint(model, volatility, 105 * petbutane + 3 * reformate + 12 * petcrknaphtha >= 17 * petrol)
@constraint(model, sulfur, 0.12 * dslcrknaphtha + 0.76 * dslcrkgasoil + 0.03 * dslgasoil <= 0.05 * diesel)

# final product
@constraint(model, butane == refbutane + disbutane - petbutane)
@constraint(model, petrol == petbutane + petcrknaphtha + reformate)
@constraint(model, diesel == dslcrkgasoil + dslcrknaphtha + dslgasoil)
@constraint(model, heating == hocrkgasoil + hocrknaphtha + hogasoil)


# objective
crudeCost = 2.1 * (crude1 + crude2)
disCost = 0 * disbutane + 4.18 * naphtha + 0.6 * residue + 2.04 * gasoil
@objective(model, Min, crudeCost + disCost)

println(model)

Min 2.1 crude1 + 2.1 crude2 + 4.18 naphtha + 0.6 residue + 2.04 gasoil
Subject to
 crknaphtha - petcrknaphtha - hocrknaphtha - dslcrknaphtha == 0.0
 crkgasoil - hocrkgasoil - dslcrkgasoil == 0.0
 gasoil - hogasoil - dslgasoil == 0.0
 butane - refbutane - disbutane + petbutane == 0.0
 petrol - petbutane - petcrknaphtha - reformate == 0.0
 diesel - dslcrkgasoil - dslcrknaphtha - dslgasoil == 0.0
 heating - hocrkgasoil - hocrknaphtha - hogasoil == 0.0
 butane >= 20000.0
 petrol >= 40000.0
 diesel >= 30000.0
 heating >= 42000.0
 octane_value : 120 petbutane + 100 reformate + 74 petcrknaphtha - 94 petrol >= 0.0
 volatility : 105 petbutane + 3 reformate + 12 petcrknaphtha - 17 petrol >= 0.0
 disbutane - 0.03 crude1 - 0.05 crude2 <= 0.0
 naphtha - 0.15 crude1 - 0.2 crude2 <= 0.0
 residue - 0.15 crude1 - 0.1 crude2 <= 0.0
 gasoil - 0.4 crude1 - 0.35 crude2 <= 0.0
 refbutane - 0.15 naphtha <= 0.0
 reformate - 0.85 naphtha <= 0.0
 crknaphtha - 0.4 residue <= 0.0
 crkgasoil - 0.35 residue <= 0.0


In [214]:
optimize!(model)

In [215]:
JuMP.termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [216]:
JuMP.getobjectivevalue(model)

1.1754e6

In [217]:
value.(crude1), value.(crude2)

(0.0, 440000.00000000006)

In [229]:
round.([value.(butane), value.(petrol), value.(diesel), value.(heating)], digits = 1)

4-element Array{Float64,1}:
 20000.0
 40000.0
 30000.0
 42000.0

In [219]:
value.(disbutane), value.(naphtha), value.(gasoil), value.(residue)

(22000.000000000004, 29999.999999999993, 49999.99999999997, 39999.99999999999)

In [220]:
value.(refbutane), value.(reformate)

(4499.999999999999, 25499.999999999985)

In [221]:
( 120 * value.(petbutane) + 100 * value.(reformate) + 74 * value.(petcrknaphtha) ) / value.(petrol)

98.04999999999998

In [222]:
( 60 * value.(petbutane) + 2.6 * value.(reformate) + 4.1 * value.(petcrknaphtha) ) / value.(petrol)

12.227499999999994

In [223]:
( 105 * value.(petbutane) + 3 * value.(reformate) + 12 * value.(petcrknaphtha) ) / value.(petrol)

21.374999999999996

In [224]:
( 0.12 * value.(dslcrknaphtha) + 0.76 * value.(dslcrkgasoil) + 0.03 * value.(dslgasoil) ) / value.(diesel)

0.049999999999999996